In [1]:
## Load all libraries and setup the synthetic problem
from library.Mag import Mag, ProblemSetter, MathUtils, Simulator, DataIO
import numpy as np
from SimPEG import PF, Utils, Mesh, Maps
from SimPEG import Utils
from SimPEG.Utils import mkvc
import SimPEG.PF as PF
import scipy as sp
import re
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import ipywidgets as widgets

# Load data and topo and build default model
workDir = r"C:\Dianne\Data\Public\GBC_Search\Phase2\Mag\AnomalousMagneticField_updatedApr17"
fName = '\MAG_UTM09.grd'

dataGrid = DataIO.loadGRDFile(workDir+fName, plotIt=False)

# 2.2.4.	Depth to Source#

Here we apply the tilt-depth method discussed in Notebook 2.1.4. to estimate depth to source in the Search Phase II magnetic data. Depth can be accurately derived from the tilt angle where contacts between magnetically contrasted geological units are vertical. Because of the typically complex nature of true geological contacts, calculated depths calculated from real magnetic data sets are usually approximations. The results should always be considered in light of expected geology, and ideally should be interpreted alongside geology, topographic data, and other geophysical data and models if these data are available.

<!-- <img src="./images/SearchQuestII.png"> -->


## Total magnetic intensity and area selection

Below is the gridded total magnetic intensity data from the Search Phase II project. The magnetic data is interpolated onto a 50 $m^2$ grid, plotted using coordinate system NAD 83 UTM Zone 9.  

A localized area or feature of interest can be windowed out for investigation using the tilt-depth method. The data 'window' shown in the left map (black outline) can be resized and moved anywhere over the data using the slider bars.


In [2]:
selection = ProblemSetter.setDataExtentWidget(dataGrid)
display(selection)

interactive(children=(FloatSlider(value=669500.0, continuous_update=False, description='East', max=816075.0, min=571025.0, step=500.0), FloatSlider(value=6069500.0, continuous_update=False, description='North', max=6152775.0, min=5881025.0, step=10.0), FloatSlider(value=40000.0, continuous_update=False, description='Width', max=100000.0, min=1000.0, step=1000.0), FloatSlider(value=40000.0, continuous_update=False, description='Height', max=100000.0, min=1000.0, step=1000.0), Output()), _dom_classes=('widget-interact',))

## Tilt angle


In [5]:
polylines, attributes = MathUtils.estimateDepth(selection.result)

# Export to shapefile
DataIO.exportShapefile(polylines, attributes, EPSGCode=26909, fileName='Tilt2Depth', label='AverageDepth')

## Tilt-depth

In [4]:
scatterData = {}
scatterData['x'] = np.vstack(polylines)[:,0]
scatterData['y'] = np.vstack(polylines)[:,1]
scatterData['z'] = 1
scatterData['c'] = np.concatenate(attributes)
scatterData['cmap'] = 'jet_r'
scatterData['clim'] = [0, 250]

view = Simulator.dataHillsideWidget(selection.result,figName='Tilt2Depth', scatterData=scatterData)
display(view)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='SunAzimuth', max=360.0, step=5.0), FloatSlider(value=15.0, continuous_update=False, description='SunAngle', max=90.0, step=5.0), FloatSlider(value=0.3, continuous_update=False, description='ColorTransp', max=1.0), FloatSlider(value=1.0, continuous_update=False, description='HSTransp', max=1.0), FloatSlider(value=1.0, continuous_update=False, description='vScale', max=4.0, min=1.0, step=1.0), IntSlider(value=50, continuous_update=False, description='Contours', min=10, step=10), Dropdown(description='ColorMap', index=3, options=('viridis', 'plasma', 'magma', 'RdBu_r', 'Greys_r', 'jet', 'hsv', 'rainbow', 'pink', 'bone', 'hsv', 'nipy_spectral'), value='RdBu_r'), FloatRangeSlider(value=(-778.33984375, 1930.2952880859375), continuous_update=False, description='Color Range', max=1930.2952880859375, min=-778.33984375, readout_format='.1f', step=1.0), Dropdown(description='Color Normalization', index=1, options=('Linear', 'HistEqualized'), value='HistEqualized'), ToggleButton(value=False, description='Export geoTiff', icon='check', tooltip='Description'), Output()), _dom_classes=('widget-interact',))